In [10]:
import pandas as pd

In [11]:
import numpy as np

In [12]:
movies = pd.read_csv("Data/movies.dat", sep=";", index_col=0)

In [13]:
movies.head()

,title,genre,movieId,imdbId,year
0,;title;genre;movieId;imdbI,0,0,0,;yea
1,1;Toy Story ;Animation|Children's|Comedy;1;11470,0,1,114709,;199
2,2;Jumanji ;Adventure|Children's|Fantasy;2;11349,0,2,113497,;199
3,3;Grumpier Old Men ;Comedy|Romance;3;11322,0,3,113228,;199
4,4;Waiting to Exhale ;Comedy|Drama;4;11488,0,4,114885,;199


In [14]:
movies = movies.loc[:, ["title", "genre", "year"]]
movies.head()

,title,genre,year
0,;title;genre;movieId;imdbI,0,;yea
1,1;Toy Story ;Animation|Children's|Comedy;1;11470,0,;199
2,2;Jumanji ;Adventure|Children's|Fantasy;2;11349,0,;199
3,3;Grumpier Old Men ;Comedy|Romance;3;11322,0,;199
4,4;Waiting to Exhale ;Comedy|Drama;4;11488,0,;199


In [15]:
genre_list = "Action*Adventure*Animation*Children's*Comedy*Crime*Documentary*Drama*Fantasy*Film-Noir*Horror*Musical*Mystery*Romance*Sci-Fi*Thriller*War*Western".split("*")

In [16]:
genre_list

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [17]:
# for i, g in enumerate(genre_list):
#     movies[g] = movies.genre.apply(lambda x: 1 if g in x.split("|") else 0)

In [19]:
movies["tags"] = (
    movies["title"] + " " +
    movies["year"].apply(str) + " " +
    movies["genre"].apply(lambda x: str(x).replace("|", " "))
)


In [21]:
print(movies["genre"].head())
print(movies["genre"].apply(type).unique())


0    0
1    0
2    0
3    0
4    0
Name: genre, dtype: int64
[<class 'int'>]


In [22]:
movies["genre"] = movies["genre"].apply(str)


In [23]:
# movies.drop(axis=1, labels=["genre"], inplace=True)
movies.head()

,title,genre,year,tags
0,;title;genre;movieId;imdbI,0,;yea,;title;genre;movieId;imdbI ;yea 0
1,1;Toy Story ;Animation|Children's|Comedy;1;11470,0,;199,1;Toy Story ;Animation|Children's|Comedy;1;114...
2,2;Jumanji ;Adventure|Children's|Fantasy;2;11349,0,;199,2;Jumanji ;Adventure|Children's|Fantasy;2;1134...
3,3;Grumpier Old Men ;Comedy|Romance;3;11322,0,;199,3;Grumpier Old Men ;Comedy|Romance;3;11322 ;199 0
4,4;Waiting to Exhale ;Comedy|Drama;4;11488,0,;199,4;Waiting to Exhale ;Comedy|Drama;4;11488 ;199 0


In [24]:
movies.shape

(3884, 4)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
tf_idf = TfidfVectorizer()

In [27]:
X = tf_idf.fit_transform(movies.tags)

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
sim_mat = cosine_similarity(X, X)

In [29]:
pd.DataFrame(sim_mat).head()

,0,1,2,3,4,5,6,7,8,9,...,3874,3875,3876,3877,3878,3879,3880,3881,3882,3883
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1.000000,0.081074,0.031180,0.032917,0.030901,0.012937,0.039454,0.070982,0.012046,...,0.015351,0.015293,0.103828,0.000000,0.0,0.014492,0.000000,0.000000,0.000000,0.000000
2,0.0,0.081074,1.000000,0.010730,0.011328,0.010634,0.013432,0.013578,0.133649,0.012507,...,0.000000,0.000000,0.082485,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.031180,0.010730,1.000000,0.029711,0.027890,0.011676,0.084265,0.009395,0.010872,...,0.013856,0.013803,0.000000,0.000000,0.0,0.013080,0.000000,0.000000,0.000000,0.000000
4,0.0,0.032917,0.011328,0.029711,1.000000,0.029444,0.012327,0.037594,0.009918,0.011478,...,0.014628,0.025523,0.000000,0.010365,0.0,0.013809,0.010251,0.011007,0.010454,0.010841


In [30]:
def kSimilarMovies(index, k = 10):
    indices = []
    score_series = pd.Series(sim_mat[index-1]).sort_values(ascending = False)
    similarities = list(score_series.iloc[1:k+1])
    top_k_indexes = list(score_series.iloc[1:k+1].index)
    
    for i in top_k_indexes:
        indices.append(list(movies.index)[i])
    return indices

In [31]:
# movies = pd.read_csv("Data/movies.dat", sep=";", index_col=0)
# movies = movies.loc[:, ["title", "genre", "year"]]
# movies["tags"] = movies.title + " " + movies.year.apply(str) + " " + movies.genre.apply(lambda x: x.replace("|", " "))
# tf_idf = TfidfVectorizer()
# X = tf_idf.fit_transform(movies.tags)
# sim_mat = cosine_similarity(X, X)

# indices = pd.Series(movies.index)

# def kSimilarMovies(index, k = 10):
#     indices = []
#     score_series = pd.Series(sim_mat[index-1]).sort_values(ascending = False)
#     similarities = list(score_series.iloc[1:k+1])
#     top_k_indexes = list(score_series.iloc[1:k+1].index)

#     for i in top_k_indexes:
#         indices.append(list(movies.index)[i])
#     return indices

In [33]:
print(movies.index)  # Shows the index (labels) of the DataFrame
print(movies.head())  # Preview the DataFrame structure


Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       3874, 3875, 3876, 3877, 3878, 3879, 3880, 3881, 3882, 3883],
      dtype='int64', length=3884)
                                              title genre  year  \
0                        ;title;genre;movieId;imdbI     0  ;yea   
1  1;Toy Story ;Animation|Children's|Comedy;1;11470     0  ;199   
2   2;Jumanji ;Adventure|Children's|Fantasy;2;11349     0  ;199   
3        3;Grumpier Old Men ;Comedy|Romance;3;11322     0  ;199   
4         4;Waiting to Exhale ;Comedy|Drama;4;11488     0  ;199   

                                                tags  
0                  ;title;genre;movieId;imdbI ;yea 0  
1  1;Toy Story ;Animation|Children's|Comedy;1;114...  
2  2;Jumanji ;Adventure|Children's|Fantasy;2;1134...  
3  3;Grumpier Old Men ;Comedy|Romance;3;11322 ;199 0  
4   4;Waiting to Exhale ;Comedy|Drama;4;11488 ;199 0  


In [34]:
for i in kSimilarMovies(1):
    print(movies.iloc[i]["title"])


3937;Runaway ;Sci-Fi|Thriller;3937;8802
3936;Phantom of the Opera, The ;Drama|Thriller;3936;3626
12;Dracula: Dead and Loving It ;Comedy|Horror;12;11289
3952;Contender, The ;Drama|Thriller;3952;20887
1;Toy Story ;Animation|Children's|Comedy;1;11470
2;Jumanji ;Adventure|Children's|Fantasy;2;11349
3;Grumpier Old Men ;Comedy|Romance;3;11322
4;Waiting to Exhale ;Comedy|Drama;4;11488
5;Father of the Bride Part II ;Comedy;5;11304
6;Heat ;Action|Crime|Thriller;6;11327


In [36]:
movies.iloc[858]

title           868;Death in Brunswick ;Comedy;868;10169
genre                                                  0
year                                                ;199
tags     868;Death in Brunswick ;Comedy;868;10169 ;199 0
Name: 858, dtype: object

In [37]:
list(movies.index)[1]

1